# Setup PostgreSQL - World Cup ETL

**Auteur** : Short Kings Team 
**Date** : 16/12/2025

## Objectif
Création de la base de données PostgreSQL avec tables, partitions et index.

# 1. Installation des dépendances

In [1]:
# Installer psycopg2 si nécessaire
!pip install psycopg2-binary

In [2]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
from dotenv import load_dotenv
import os
print(f"psycopg2 version: {psycopg2.__version__}")

psycopg2 version: 2.9.11 (dt dec pq3 ext lo64)


# 2. Configuration de connexion
```
DB_USER=ton_user_render
DB_PASSWORD=ton_password_render
DB_HOST=oregon-postgres.render.com
DB_PORT=5432
DB_NAME=ton_dbname_render
il va falloir que je vous donne mes credentials
```


In [3]:
# Charger les variables d'environnement depuis .env
load_dotenv()

# Récupération des variables d'environnement
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME')

# Construire l'URL de connexion
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Vérification
print(f"Host: {DB_HOST}")
print(f"Database: {DB_NAME}")
print(f"User: {DB_USER}")
print(f"Password: {'chargé' if DB_PASSWORD else 'manquant'}")

Host: dpg-d50mcnnfte5s73cqqbag-a.frankfurt-postgres.render.com
Database: worldcup_db_bpk8
User: worldcup_db_bpk8_user
Password: chargé


# 3. Création de la base de données
inutile avec Render

In [4]:
## Connexion au serveur PostgreSQL (sans base spécifique)
#conn = psycopg2.connect(**DB_CONFIG)
#conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
#cursor = conn.cursor()

# Vérifier si la base existe déjà
#cursor.execute("SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s", (DB_NAME,))
#exists = cursor.fetchone()

#if not exists:
    #cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(DB_NAME)))
    #print(f"Base de données '{DB_NAME}' créée avec succès!")
#else:
    #print(f"La base de données '{DB_NAME}' existe déjà.")

#cursor.close()
#conn.close()

# 4. Fonctions utilitaires

In [5]:
def get_engine():
    """Retourne un engine SQLAlchemy (pour pandas)"""
    return create_engine(DATABASE_URL)
    
def get_connection():
    """Retourne une connexion à la base de données"""
    return psycopg2.connect(DATABASE_URL)

def execute_sql(query, fetch=False, raise_on_error=True):
    """
    Exécute une requête SQL.
    
    Args:
        query: Requête SQL à exécuter
        fetch: Si True, retourne les résultats
        raise_on_error: Si True, lève une exception en cas d'erreur
    
    Returns:
        Les résultats si fetch=True, True si succès, False si erreur (et raise_on_error=False)
    """
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        if fetch:
            return cursor.fetchall()
        print("Requête exécutée avec succès")
        return True
    except Exception as e:
        conn.rollback()
        if raise_on_error:
            raise
        print(f"Erreur: {e}")
        return False
    finally:
        cursor.close()
        conn.close()

# Test connexion
try:
    conn = get_connection()
    print(f"Connecté à '{DB_NAME}'")
    conn.close()
except Exception as e:
    print(f"Erreur de connexion: {e}")

Connecté à 'worldcup_db_bpk8'


# 5. Création des tables TEAMS et STADIUMS

In [6]:
sql_schema = """
-- Suppression si existe (ordre important pour les FK)
DROP TABLE IF EXISTS matches CASCADE;
DROP TABLE IF EXISTS stadiums CASCADE;
DROP TABLE IF EXISTS teams CASCADE;

-- 1. Table teams (référentiel)
CREATE TABLE teams (
    id_team         SERIAL PRIMARY KEY,
    nom_standard    VARCHAR(50) NOT NULL UNIQUE,
    confederation   VARCHAR(20),
    aliases         JSONB DEFAULT '[]'::jsonb
);

CREATE INDEX idx_teams_aliases ON teams USING GIN (aliases);

COMMENT ON TABLE teams IS 'Référentiel des équipes nationales FIFA - World Cup 1930-2022';
COMMENT ON COLUMN teams.nom_standard IS 'Nom standardisé de l''équipe (ex: Germany, France)';
COMMENT ON COLUMN teams.confederation IS 'Confédération (UEFA, CONMEBOL, CAF, AFC, CONCACAF, OFC)';
COMMENT ON COLUMN teams.aliases IS 'Variantes historiques du nom en JSON (ex: ["West Germany", "RFA"])';

-- 2. Table stadiums (référentiel)
CREATE TABLE stadiums (
    id_stadium      SERIAL PRIMARY KEY,
    nom             VARCHAR(100) NOT NULL,
    ville           VARCHAR(100),
    pays            VARCHAR(50),
    capacite        INTEGER
);

COMMENT ON TABLE stadiums IS 'Référentiel des stades de Coupe du Monde';
"""

execute_sql(sql_schema)
print("Tables 'teams' et 'stadiums' créées avec index et commentaires")

Requête exécutée avec succès
Tables 'teams' et 'stadiums' créées avec index et commentaires


# 6. Création de la table MATCHES (partitionnée)

In [7]:
sql_matches = """
-- Table matches (partitionnée par édition)
CREATE TABLE matches (
    id_match        SERIAL,
    home_team_id    INTEGER NOT NULL REFERENCES teams(id_team),
    away_team_id    INTEGER NOT NULL REFERENCES teams(id_team),
    home_result     INTEGER NOT NULL CHECK (home_result >= 0),
    away_result     INTEGER NOT NULL CHECK (away_result >= 0),
    result          VARCHAR(20) NOT NULL CHECK (result IN ('home_team', 'away_team', 'draw')),
    extra_time      BOOLEAN DEFAULT FALSE,
    penalties       BOOLEAN DEFAULT FALSE,
    replay          BOOLEAN DEFAULT FALSE,
    date            DATE,  -- NULLABLE: dates manquantes pour matchs 1930-2010
    round           VARCHAR(50) NOT NULL,
    city            VARCHAR(100),
    stadium_id      INTEGER REFERENCES stadiums(id_stadium),
    edition         INTEGER NOT NULL,
    
    PRIMARY KEY (id_match, edition)
) PARTITION BY RANGE (edition);

-- Index partiels pour gérer l'unicité avec dates NULL
-- Cas 1: matchs avec date connue
CREATE UNIQUE INDEX idx_unique_match_with_date 
ON matches (home_team_id, away_team_id, date, edition) 
WHERE date IS NOT NULL;

-- Cas 2: matchs sans date - INDEX NON-UNIQUE (performance uniquement)
-- Note: L'index unique a été supprimé car il bloque les matchs légitimes
-- (séries de qualifications, matchs rejoués, etc.)
CREATE INDEX idx_matches_teams_round_edition 
ON matches (home_team_id, away_team_id, round, edition) 
WHERE date IS NULL;

-- Index de performance
CREATE INDEX idx_matches_home_team ON matches (home_team_id);
CREATE INDEX idx_matches_away_team ON matches (away_team_id);
CREATE INDEX idx_matches_edition ON matches (edition);
CREATE INDEX idx_matches_date ON matches (date) WHERE date IS NOT NULL;
CREATE INDEX idx_matches_round ON matches (round);
CREATE INDEX idx_matches_result ON matches (result);
CREATE INDEX idx_matches_city ON matches (city) WHERE city IS NOT NULL;

-- Index composite pour requêtes fréquentes
CREATE INDEX idx_matches_team_edition ON matches (home_team_id, edition);
CREATE INDEX idx_matches_away_team_edition ON matches (away_team_id, edition);

-- Commentaires
COMMENT ON TABLE matches IS 'Historique des matchs de Coupe du Monde FIFA 1930-2022';
COMMENT ON COLUMN matches.date IS 'Date du match (NULL pour certains matchs historiques 1930-2010)';
COMMENT ON COLUMN matches.result IS 'Résultat: home_team, away_team, ou draw';
COMMENT ON COLUMN matches.extra_time IS 'Match décidé en prolongation';
COMMENT ON COLUMN matches.penalties IS 'Match décidé aux tirs au but';
COMMENT ON COLUMN matches.replay IS 'Match rejoué';
COMMENT ON COLUMN matches.round IS 'Phase: Preliminary, Group Stage, Round of 16, Quarter-finals, Semi-finals, Third Place, Final';
COMMENT ON COLUMN matches.edition IS 'Année de l''édition (clé de partitionnement)';
"""

execute_sql(sql_matches)
print("Table 'matches' créée avec date NULLABLE et index de performance")

Requête exécutée avec succès
Table 'matches' créée avec date NULLABLE et index de performance


In [8]:
# Création des partitions par édition
editions = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 
            1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 
            2010, 2014, 2018, 2022, 2030]  # 2030 comme borne supérieure

for i, year in enumerate(editions[:-1]):
    next_year = editions[i + 1]
    sql_partition = f"""
    CREATE TABLE IF NOT EXISTS matches_{year} PARTITION OF matches
        FOR VALUES FROM ({year}) TO ({next_year});
    """
    execute_sql(sql_partition)
    print(f"  Partition 'matches_{year}' créée")

Requête exécutée avec succès
  Partition 'matches_1930' créée
Requête exécutée avec succès
  Partition 'matches_1934' créée
Requête exécutée avec succès
  Partition 'matches_1938' créée
Requête exécutée avec succès
  Partition 'matches_1950' créée
Requête exécutée avec succès
  Partition 'matches_1954' créée
Requête exécutée avec succès
  Partition 'matches_1958' créée
Requête exécutée avec succès
  Partition 'matches_1962' créée
Requête exécutée avec succès
  Partition 'matches_1966' créée
Requête exécutée avec succès
  Partition 'matches_1970' créée
Requête exécutée avec succès
  Partition 'matches_1974' créée
Requête exécutée avec succès
  Partition 'matches_1978' créée
Requête exécutée avec succès
  Partition 'matches_1982' créée
Requête exécutée avec succès
  Partition 'matches_1986' créée
Requête exécutée avec succès
  Partition 'matches_1990' créée
Requête exécutée avec succès
  Partition 'matches_1994' créée
Requête exécutée avec succès
  Partition 'matches_1998' créée
Requête 

# 7. Vérification de la structure

In [9]:
# Lister les tables
sql_tables = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public'
ORDER BY table_name;
"""

engine = get_engine()
df_tables = pd.read_sql(sql_tables, engine)

print("Tables créées :")
df_tables

Tables créées :


,table_name
0,matches
1,matches_1930
2,matches_1934
3,matches_1938
4,matches_1950
5,matches_1954
6,matches_1958
7,matches_1962
8,matches_1966
9,matches_1970


In [10]:
# Structure de la table teams
sql_columns = """
SELECT column_name, data_type, is_nullable, column_default
FROM information_schema.columns
WHERE table_name = 'teams'
ORDER BY ordinal_position;
"""

engine = get_engine()
df_teams = pd.read_sql(sql_columns, engine)

print("Structure table 'teams' :")
df_teams

Structure table 'teams' :


,column_name,data_type,is_nullable,column_default
0,id_team,integer,NO,nextval('teams_id_team_seq'::regclass)
1,nom_standard,character varying,NO,None
2,confederation,character varying,YES,None
3,aliases,jsonb,YES,'[]'::jsonb


In [11]:
# Structure de la table matches
sql_columns = """
SELECT column_name, data_type, is_nullable, column_default
FROM information_schema.columns
WHERE table_name = 'matches'
ORDER BY ordinal_position;
"""

engine = get_engine()
df_matches = pd.read_sql(sql_columns, engine)

print("Structure table 'matches' :")
df_matches

Structure table 'matches' :


,column_name,data_type,is_nullable,column_default
0,id_match,integer,NO,nextval('matches_id_match_seq'::regclass)
1,home_team_id,integer,NO,None
2,away_team_id,integer,NO,None
3,home_result,integer,NO,None
4,away_result,integer,NO,None
5,result,character varying,NO,None
6,extra_time,boolean,YES,false
7,penalties,boolean,YES,false
8,replay,boolean,YES,false
9,date,date,YES,None


In [12]:
# Lister les index
sql_indexes = """
SELECT indexname, tablename
FROM pg_indexes
WHERE schemaname = 'public'
ORDER BY tablename, indexname;
"""

engine = get_engine()
df_indexes = pd.read_sql(sql_indexes, engine)

print(f"Index créés ({len(df_indexes)}) :")
df_indexes

Index créés (280) :


,indexname,tablename
0,idx_matches_away_team,matches
1,idx_matches_away_team_edition,matches
2,idx_matches_city,matches
3,idx_matches_date,matches
4,idx_matches_edition,matches
...,...,...
275,matches_2022_round_idx,matches_2022
276,stadiums_pkey,stadiums
277,idx_teams_aliases,teams
278,teams_nom_standard_key,teams


# 8. Création des vues analytiques

Vues pré-calculées pour faciliter les requêtes d'analyse.

In [13]:
# Vue 1: Statistiques globales par équipe
sql_view_team_stats = """
CREATE OR REPLACE VIEW v_team_stats AS
WITH team_matches AS (
    SELECT
        t.id_team,
        t.nom_standard,
        t.confederation,
        m.edition,
        -- Buts marqués/encaissés selon position (home/away)
        CASE WHEN m.home_team_id = t.id_team THEN m.home_result ELSE m.away_result END AS goals_for,
        CASE WHEN m.home_team_id = t.id_team THEN m.away_result ELSE m.home_result END AS goals_against,
        -- Résultat du match pour cette équipe
        CASE
            WHEN (m.home_team_id = t.id_team AND m.result = 'home_team')
                 OR (m.away_team_id = t.id_team AND m.result = 'away_team') THEN 'win'
            WHEN m.result = 'draw' THEN 'draw'
            ELSE 'loss'
        END AS match_result,
        m.round,
        m.extra_time,
        m.penalties
    FROM teams t
    JOIN matches m ON t.id_team IN (m.home_team_id, m.away_team_id)
    WHERE m.round != 'Preliminary'  -- Exclure les matchs de qualification
)
SELECT
    id_team,
    nom_standard,
    confederation,
    COUNT(*) AS total_matches,
    COUNT(DISTINCT edition) AS editions,
    SUM(CASE WHEN match_result = 'win' THEN 1 ELSE 0 END) AS wins,
    SUM(CASE WHEN match_result = 'draw' THEN 1 ELSE 0 END) AS draws,
    SUM(CASE WHEN match_result = 'loss' THEN 1 ELSE 0 END) AS losses,
    SUM(goals_for) AS goals_for,
    SUM(goals_against) AS goals_against,
    SUM(goals_for) - SUM(goals_against) AS goal_difference,
    ROUND(100.0 * SUM(CASE WHEN match_result = 'win' THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0), 1) AS win_rate,
    ROUND(AVG(goals_for)::DECIMAL, 2) AS avg_goals_scored,
    ROUND(AVG(goals_against)::DECIMAL, 2) AS avg_goals_conceded,
    SUM(CASE WHEN goals_against = 0 THEN 1 ELSE 0 END) AS clean_sheets
FROM team_matches
GROUP BY id_team, nom_standard, confederation;

COMMENT ON VIEW v_team_stats IS 'Statistiques globales par équipe (hors matchs Preliminary)';
"""

execute_sql(sql_view_team_stats)
print("Vue 'v_team_stats' créée")

Requête exécutée avec succès
Vue 'v_team_stats' créée


In [14]:
# Vue 2: Statistiques par équipe ET par édition
sql_view_team_by_edition = """
CREATE OR REPLACE VIEW v_team_by_edition AS
WITH team_matches AS (
    SELECT
        t.id_team,
        t.nom_standard,
        t.confederation,
        m.edition,
        CASE WHEN m.home_team_id = t.id_team THEN m.home_result ELSE m.away_result END AS goals_for,
        CASE WHEN m.home_team_id = t.id_team THEN m.away_result ELSE m.home_result END AS goals_against,
        CASE
            WHEN (m.home_team_id = t.id_team AND m.result = 'home_team')
                 OR (m.away_team_id = t.id_team AND m.result = 'away_team') THEN 'win'
            WHEN m.result = 'draw' THEN 'draw'
            ELSE 'loss'
        END AS match_result,
        m.round
    FROM teams t
    JOIN matches m ON t.id_team IN (m.home_team_id, m.away_team_id)
    WHERE m.round != 'Preliminary'
)
SELECT
    id_team,
    nom_standard,
    confederation,
    edition,
    COUNT(*) AS matches,
    SUM(CASE WHEN match_result = 'win' THEN 1 ELSE 0 END) AS wins,
    SUM(CASE WHEN match_result = 'draw' THEN 1 ELSE 0 END) AS draws,
    SUM(CASE WHEN match_result = 'loss' THEN 1 ELSE 0 END) AS losses,
    SUM(goals_for) AS goals_for,
    SUM(goals_against) AS goals_against,
    SUM(goals_for) - SUM(goals_against) AS goal_diff,
    ROUND(100.0 * SUM(CASE WHEN match_result = 'win' THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0), 1) AS win_rate,
    -- Meilleur tour atteint (ordre: Group Stage < R16 < QF < SF < Final)
    MAX(CASE round
        WHEN 'Final' THEN 6
        WHEN 'Third Place' THEN 5
        WHEN 'Semi-finals' THEN 5
        WHEN 'Quarter-finals' THEN 4
        WHEN 'Round of 16' THEN 3
        WHEN 'Group Stage' THEN 2
        ELSE 1
    END) AS best_round_rank
FROM team_matches
GROUP BY id_team, nom_standard, confederation, edition;

COMMENT ON VIEW v_team_by_edition IS 'Performance de chaque équipe par édition de Coupe du Monde';
"""

execute_sql(sql_view_team_by_edition)
print("Vue 'v_team_by_edition' créée")

Requête exécutée avec succès
Vue 'v_team_by_edition' créée


In [15]:
sql_view_h2h = """
CREATE OR REPLACE VIEW v_head_to_head AS
WITH match_details AS (
    SELECT
        -- Normaliser les paires (toujours alphabétique) avec LEAST/GREATEST
        LEAST(t1.nom_standard, t2.nom_standard) AS team_a,
        GREATEST(t1.nom_standard, t2.nom_standard) AS team_b,
        m.edition,
        m.round,
        m.date,
        m.city,
        t1.nom_standard AS home_team,
        t2.nom_standard AS away_team,
        m.home_result,
        m.away_result,
        m.result,
        m.extra_time,
        m.penalties,
        -- Vainqueur
        CASE
            WHEN m.result = 'home_team' THEN t1.nom_standard
            WHEN m.result = 'away_team' THEN t2.nom_standard
            ELSE 'Draw'
        END AS winner
    FROM matches m
    JOIN teams t1 ON m.home_team_id = t1.id_team
    JOIN teams t2 ON m.away_team_id = t2.id_team
    WHERE m.round != 'Preliminary'
)
SELECT
    team_a,
    team_b,
    COUNT(*) AS total_matches,
    -- Victoires de team_a
    SUM(CASE WHEN winner = team_a THEN 1 ELSE 0 END) AS team_a_wins,
    -- Victoires de team_b
    SUM(CASE WHEN winner = team_b THEN 1 ELSE 0 END) AS team_b_wins,
    -- Nuls
    SUM(CASE WHEN winner = 'Draw' THEN 1 ELSE 0 END) AS draws,
    -- Buts marqués par team_a
    SUM(CASE 
        WHEN home_team = team_a THEN home_result 
        ELSE away_result 
    END) AS team_a_goals,
    -- Buts marqués par team_b
    SUM(CASE 
        WHEN home_team = team_b THEN home_result 
        ELSE away_result 
    END) AS team_b_goals,
    -- Matchs avec prolongation
    SUM(CASE WHEN extra_time THEN 1 ELSE 0 END) AS extra_time_matches,
    -- Matchs aux tirs au but
    SUM(CASE WHEN penalties THEN 1 ELSE 0 END) AS penalty_matches,
    -- Liste des éditions
    STRING_AGG(DISTINCT edition::TEXT, ', ' ORDER BY edition::TEXT) AS editions
FROM match_details
GROUP BY team_a, team_b;

COMMENT ON VIEW v_head_to_head IS 'Historique des confrontations directes entre paires d''équipes';
"""

execute_sql(sql_view_h2h)
print("Vue 'v_head_to_head' créée")

Requête exécutée avec succès
Vue 'v_head_to_head' créée
